In [1]:
import requests
import pandas as pd
import numpy as np
import unicodedata
#Seccion de API google Sheet
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build


## **Seccion 1:** Descargue de datos
*****
Se descargaran los datos directamente de la  pagina [datos.gov](https://datos.gov.co/api/views/eh75-8ah6/rows.csv?accessType=DOWNLOAD), donde se almacenaran bajo el nombre de `datos_sinLimpiar.csv`

**Nota:** Esto puede tomar un tiempo, por ello tomaremos una muestra equivalente al 50%

* Descargar los datos directamente de la pagina web

In [ ]:
url = "https://datos.gov.co/api/views/eh75-8ah6/rows.csv?accessType=DOWNLOAD"

#Extraccion de los datos dado url
response = requests.get(url)

if response.status_code == 200:
  with open('datos_sinLimpiar.csv','wb') as archivo:
    archivo.write(response.content)


#### Comprobacion de los datos descargados
****




In [6]:
datos = pd.read_csv('Content/datos_sinLimpiar.csv')
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,FECHA_DESPACHO,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS
0,T.T. DE BOGOTÁ NORTE,MICROBUS,LUJO,11001,15759,03/01/2021 12:00:00 AM,8,TRANSITO,2,0
1,T.T. DE BOGOTÁ SALITRE,BUS,LUJO,11001,76001,03/12/2021 12:00:00 AM,11,ORIGEN,4,45
2,T.T. DE CALI,BUSETA,BASICO,76001,52378,01/09/2021 12:00:00 AM,6,ORIGEN,1,3
3,T.T. DE MELGAR,BUS,BASICO,73001,11001,08/13/2021 12:00:00 AM,16,TRANSITO,1,10
4,T.T. DE MELGAR,MICROBUS,BASICO,73449,25488,02/28/2021 12:00:00 AM,12,ORIGEN,1,1


##### Datos de muestreo
****
Dado a al cantidad masiva de datos se usara una muestra aleatoria de estos a la cual se le aplicara la limpieza de los datos

In [7]:
print(f"Longitud de datos originales {len(datos)}")
#para este caso tomaremos el 50% de los datos totales
datos = datos[:int(len(datos)*0.5)]
print(f"longitud de la muestra {len(datos)}")

Longitud de datos originales 21613532
longitud de la muestra 10806766


### Subseccion 1.1: Descargue de datos de diviPola
****
Se extraeran los datos del codigo municipal para su correcta traduccion en los `datos_sinLimpiar.csv`,donde a su vez esto se almacenaran en un arhivo ..csv llamado `diccionario_municipios.csv`

In [3]:
url = "https://datos.gov.co/api/views/gdxc-w37w/rows.csv?accessType=DOWNLOAD"

#Extraccion de los datos dado url
response = requests.get(url)

if response.status_code == 200:
  with open('Content/diccionario_municipios.csv','wb') as archivo:
    archivo.write(response.content)

#### Comprobacion de los datos descargados
****


In [8]:
dicc = pd.read_csv("Content/diccionario_municipios.csv")
dicc.head()

,Código Departamento,Código Municipio,Nombre Departamento,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada
0,05,5001.0,ANTIOQUIA,MEDELLÍN,Municipio
1,05,5002.0,ANTIOQUIA,ABEJORRAL,Municipio
2,05,5004.0,ANTIOQUIA,ABRIAQUÍ,Municipio
3,05,5021.0,ANTIOQUIA,ALEJANDRÍA,Municipio
4,05,5030.0,ANTIOQUIA,AMAGÁ,Municipio


## **Seccion 2:** Limpieza del dataset dicc
****

* DEtectando datos con elementos nulos

In [9]:
dicc.isnull().sum()

Código Departamento                                0
Código Municipio                                   5
Nombre Departamento                                5
Nombre Municipio                                   5
Tipo: Municipio / Isla / Área no municipalizada    5
dtype: int64

* Eliminando datos nulos

In [10]:
dicc=dicc.dropna()

#Comprobando
dicc.isnull().sum()

Código Departamento                                0
Código Municipio                                   0
Nombre Departamento                                0
Nombre Municipio                                   0
Tipo: Municipio / Isla / Área no municipalizada    0
dtype: int64

* Transformando codigo de municipios de float a int

In [11]:
dicc.dtypes

Código Departamento                                 object
Código Municipio                                   float64
Nombre Departamento                                 object
Nombre Municipio                                    object
Tipo: Municipio / Isla / Área no municipalizada     object
dtype: object

* Quitar tildes

In [12]:
def quitar_tildes(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')


#Quitar las tildes del texto
dicc["Nombre Municipio"] = dicc["Nombre Municipio"].apply(quitar_tildes)
datos["TERMINAL"] = datos["TERMINAL"].apply(quitar_tildes)

In [13]:
dicc['Código Municipio']=dicc['Código Municipio'].astype('int')

## **Seccion 3:** Traduccion de los datos de dicc en datos
****

* Creando la funcion de traduccion

In [14]:
diccionario=np.array(dicc.iloc[:,[1,3]])
diccionario = dict(diccionario)

In [15]:
diccionario_depa=np.array(dicc.iloc[:,[1,3]])
diccionario_depa = dict(diccionario_depa)

In [16]:
diccionario[datos['MUNICIPIO_ORIGEN_RUTA'][0]]


'BOGOTA. D.C.'

In [23]:
datos['MUNICIPIO_ORIGEN_RUTA'] = [ diccionario[dato] for dato in datos['MUNICIPIO_ORIGEN_RUTA']]
datos['MUNICIPIO_DESTINO_RUTA'] = [ diccionario[dato] for dato in datos['MUNICIPIO_DESTINO_RUTA']]


## **Seccion 4:** Transformacion de fecha a dia , mes y año
***


* Extraccion de la hora y la fecha

In [17]:
datos[['fecha','hora','tipo_hora']]=datos['FECHA_DESPACHO'].str.split(' ',expand=True)
datos.drop(columns=['FECHA_DESPACHO'],inplace=True)

* Extraccion de dia,mes y año con base a la columna recien agregada *fecha*

In [18]:
datos[['mes_despacho','dia_despacho','ano_despacho']] = datos['fecha'].str.split('/',expand=True)
datos.drop(columns=['fecha'],inplace=True)
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS,hora,tipo_hora,mes_despacho,dia_despacho,ano_despacho
0,T.T. DE BOGOTA NORTE,MICROBUS,LUJO,11001,15759,8,TRANSITO,2,0,12:00:00,AM,03,01,2021
1,T.T. DE BOGOTA SALITRE,BUS,LUJO,11001,76001,11,ORIGEN,4,45,12:00:00,AM,03,12,2021
2,T.T. DE CALI,BUSETA,BASICO,76001,52378,6,ORIGEN,1,3,12:00:00,AM,01,09,2021
3,T.T. DE MELGAR,BUS,BASICO,73001,11001,16,TRANSITO,1,10,12:00:00,AM,08,13,2021
4,T.T. DE MELGAR,MICROBUS,BASICO,73449,25488,12,ORIGEN,1,1,12:00:00,AM,02,28,2021


* **Nota:** Dado que todas las horas son iguales se eliminaran  la columna de *hora* y *tipo hora*

In [19]:
#Comrpobacion de lo mencionado
datos['hora'].unique()[0]

'12:00:00'

In [20]:
datos.drop(columns=['hora','tipo_hora'],inplace=True)
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS,mes_despacho,dia_despacho,ano_despacho
0,T.T. DE BOGOTA NORTE,MICROBUS,LUJO,11001,15759,8,TRANSITO,2,0,03,01,2021
1,T.T. DE BOGOTA SALITRE,BUS,LUJO,11001,76001,11,ORIGEN,4,45,03,12,2021
2,T.T. DE CALI,BUSETA,BASICO,76001,52378,6,ORIGEN,1,3,01,09,2021
3,T.T. DE MELGAR,BUS,BASICO,73001,11001,16,TRANSITO,1,10,08,13,2021
4,T.T. DE MELGAR,MICROBUS,BASICO,73449,25488,12,ORIGEN,1,1,02,28,2021


## **Seccion 5:** Guardando el nuevo csv
***

* Comproabcion del estado del dataset

In [24]:
datos.head()

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS,mes_despacho,dia_despacho,ano_despacho
0,T.T. DE BOGOTA NORTE,MICROBUS,LUJO,BOGOTA. D.C.,SOGAMOSO,8,TRANSITO,2,0,03,01,2021
1,T.T. DE BOGOTA SALITRE,BUS,LUJO,BOGOTA. D.C.,CALI,11,ORIGEN,4,45,03,12,2021
2,T.T. DE CALI,BUSETA,BASICO,CALI,LA CRUZ,6,ORIGEN,1,3,01,09,2021
3,T.T. DE MELGAR,BUS,BASICO,IBAGUE,BOGOTA. D.C.,16,TRANSITO,1,10,08,13,2021
4,T.T. DE MELGAR,MICROBUS,BASICO,MELGAR,NILO,12,ORIGEN,1,1,02,28,2021


* Descargue del dataset en formato .csv (nota si se descarga este pesara al rededor de 1,2GB; esto contiene toda la informacion)

* **Nota:** Si desea descargarlos coloque la palabra *si* tal y como esta escrita en esta nota

In [27]:
if input("¿Desea descargar el archivo .csv? ").lower() == 'si':
    datos.to_csv('Datos_listos.csv')

## **Seccion 6**:Cargue de datos a google sheet
****
Para esta seccion cogeremos una muestra y la almacenaremos en un libro de *google sheet* 
esto con el fin de poder crear el analisis con *power bi*

### **Seccion 6.1:** Creacion de conexion con el libro de google sheet
****

In [42]:
#informacion basica
Scopes = ['https://www.googleapis.com/auth/spreadsheets']
key = 'key.json'
id_sheet = '1v4w_c-0D8moJFnzxFxrCGN0d7Hyqni8HkgCCTD5e5F4'

#conectar
creds = None
creds = service_account.Credentials.from_service_account_file(key,scopes=Scopes)

#Generar API
service = build('sheets','v4',credentials=creds)
EXCEl = service.spreadsheets()

### **Seccion 6.2:** Cargue de los datos a google sheet
****

In [38]:
datos_list = datos.values.tolist()
columnas =list(datos.columns)

#Agregar las columnas
datos_list [0] = columnas

In [43]:
EXCEl.values().append(spreadsheetId=id _sheet,range="main_tables!A1",valueInputOption='USER_ENTERED',body={'values':datos_list[:370000]}).execute()


HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1v4w_c-0D8moJFnzxFxrCGN0d7Hyqni8HkgCCTD5e5F4/values/main_tables%21A1:append?valueInputOption=USER_ENTERED&alt=json returned "This action would increase the number of cells in the workbook above the limit of 10000000 cells.". Details: "This action would increase the number of cells in the workbook above the limit of 10000000 cells.">